# Title : Text Classification using Naive Bayes and NLP
### Name : Harsh Vardhan
#### DESCRIPTION:   Taken 20newGroup data from sklearn datasets( also taken from kaggle, due to simplecity taken it from sklearn) used Naive bayes approach to predict the class of data, and in last compare the model with inbuilt Naive Bayes classifier.

In [1]:
import numpy as np                                                  # to preform numerical operations
from sklearn import datasets                                        # to import required dataset
from sklearn import model_selection                                 # to split data into training and testing data
from nltk.corpus import stopwords                                   # for removing stopwords
import itertools                                                    # for slicing vocabulary dictionary into required length
import  pandas as pd                                                # for dataset manipulation
from sklearn.metrics import classification_report, confusion_matrix # for getting info of result precision and accuracy
from nltk.tokenize import word_tokenize                             # for tokenizing  
import math as ma



In [10]:
stop_words=set(stopwords.words('english'))                                            # get the stopwords 
news=datasets.fetch_20newsgroups()  # load data into news 
x_train,x_test,y_train,y_test=model_selection.train_test_split(news.data,news.target,test_size=0.25) # split data into test and train    

In [11]:
for i in range(len(x_train)):                                       #converting train data in lower form
    x_train[i] = x_train[i].lower()
for i in range(len(x_test)):                                        #converting test data in lower form.
    x_test[i]=x_test[i].lower()

In [12]:
print(stop_words)                                                   # Just checking what are the stopwords

{'yourself', 'from', 'further', "couldn't", 'he', 'against', 'you', 're', "hasn't", 'his', 'do', 'same', 'shan', 'll', 'mightn', 'which', 'weren', "wouldn't", 'yourselves', "you'd", "won't", 'both', 'just', 'hasn', 'did', 'then', "isn't", 'to', 'about', 'am', 'very', 'now', 'for', 'doesn', 'here', 'mustn', 'until', "doesn't", 'a', 'are', "you'll", 'because', 'doing', 'too', 'up', "wasn't", 'ours', 'haven', "you're", 'all', "mustn't", 'into', 'can', 'me', 'in', "shouldn't", "should've", 'above', "it's", 'isn', 'this', 'before', 'won', 'themselves', 'other', 'these', 'been', 'theirs', 'more', 'aren', 'and', 'during', 'again', 'between', 'where', 'under', 'our', 'o', 'itself', 'on', 'with', 'if', 'how', 'so', 'will', 'they', 'that', 'off', 'we', 'what', 'ma', 'were', 'after', 'don', 'there', 'while', 'those', 'the', 'herself', 'below', "aren't", 'or', 'whom', 'few', 'through', 'no', 'i', 'himself', 's', 'as', 'my', 'had', 'ourselves', 'such', 'have', 'she', 'but', 'their', 'when', 'them',

In [13]:
#import nltk
#nltk.download('punkt')                                             # needed to be done first time
d={}                                                                # d is going to be our vocabulary dictionary
for i in range(len(x_train)):                                       # this function filters stopwords and creates vocabulary                                                # taken one row at a time of training data
    data = x_train[i]
    words=word_tokenize(data)                                       # tokenize the data
    filtered_sentence=[ele for ele in words if ele not in stop_words] # get filtered sentence
    for ele in filtered_sentence:                                   # create vocabulary from filtered sentence    
        d[ele]=d.get(ele,0)+1                                         
new_dict={}                                                         # this will be our final vocabulary dictionary
for key,value in sorted(d.items(), key=lambda kv: kv[1],reverse=True): # put the key,value pair from d in decreasing order of value
    new_dict[key]=value
new_dict=dict(itertools.islice(new_dict.items(),6000))             # Take the top 6000 elements into final vocabulary
features=[]
for i in new_dict:
    features.append(i)

In [14]:
print(new_dict)                                                     # just checking final created vocabulary dictionary
#print(features.shape)

{'>': 139953, ',': 118856, '.': 103429, '--': 90415, ':': 84680, '(': 54308, ')': 54041, "'ax": 46454, '@': 44079, '?': 27336, "''": 24532, '<': 20460, '``': 20286, '!': 18335, '%': 14939, '$': 14178, '|': 14065, ';': 13850, '#': 12729, "'s": 12220, ']': 12190, '`': 11490, '-': 11016, "'": 10866, "n't": 10797, '[': 10778, '&': 10702, 'subject': 9181, '...': 9041, 'lines': 8840, 'organization': 8375, 'would': 7041, 'one': 6520, 'writes': 5879, 'article': 5093, '1': 4803, 'people': 4373, 'like': 4282, 'x': 4206, 'university': 4118, '*': 4008, '2': 3777, 'know': 3763, 'get': 3678, 'nntp-posting-host': 3569, 'max': 3458, 'think': 3273, 'also': 3205, 'use': 3193, '0': 3018, 'new': 2906, 'could': 2881, 'time': 2861, 'good': 2775, "'m": 2703, 'q': 2495, 'may': 2437, 'even': 2421, 'r': 2408, 'well': 2389, '3': 2316, 'see': 2312, '7': 2234, 'g': 2212, 'many': 2175, 'two': 2171, 'us': 2169, 'make': 2169, 'much': 2157, 'way': 2152, 'first': 2129, '=': 2125, '4': 2104, 'god': 2104, '+': 2076, 'dis

In [15]:
def updatedRow(words,features,i):                                   
    lst=np.array([0]*6000)                                         # Initialise the lst to be np.zeros with dimensions 1 X 6000                                                           # i will help us iterate our lst 
    for ele in words:
        if ele in features:
            lst[features.index(ele)]=words.count(ele)
    return lst

def updateXTrain(x_train,features):
    x_train_updated=pd.DataFrame(np.zeros((len(x_train),6000)))     # Intialise the np.zeros 2-D array to x_train_updated
    x_train_updated.columns=features                                # Add columns to the x_train_updated
    for i in range(len(x_train)):                                   # Now operate on each row
        curr_x=x_train[i]                                           # curr_x contains the row i content
        words=word_tokenize(curr_x)                                 # extract all words of curr_x into words
        x_train_updated.iloc[i,:]=updatedRow(words,features,i)      # replace row i with the updateRow() function's list
    return x_train_updated                                          # return x_train_updated

x_train_updated=updateXTrain(x_train,features)                      # get updated x_train_updated
x_train_updated.columns                                             # just checking the columns

Index(['>', ',', '.', '--', ':', '(', ')', ''ax', '@', '?',
       ...
       'seagate', 'cryptology', 'oj', 'eof_not_ok', 'resort', 'frequencies',
       'tobias', 'abroad', '16550', 'uart'],
      dtype='object', length=6000)

In [8]:
def fit(X_train, Y_train):                                          # This function is used to fit training data into our model
    result = {}                                                     # This dictionary is going to be useful in later calculations
    class_values = set(Y_train) 
    for current_class in class_values:                              # We create keys for all the possible classes
        result[current_class] = {}                                  # We create a dictionary as value for each key itself
        result["total"] = len(Y_train)                              # It gives total element present in our dictionary
        current_class_rows = (Y_train == current_class)             # Obtain rows for the current class
        X_train_current = X_train[current_class_rows]               # Filter the x_train for current class
        Y_train_current = Y_train[current_class_rows]               # Filter the y_train for current class
        num_features = X_train.shape[1]                             
        result[current_class]["count"] = len(Y_train_current)       # It gives total number of features of our data
        a=0                                                         # To get total number of a particular feature 
        for j in range(num_features):                               # traverse each feature
            result[current_class][j]=X_train_current[:,j].sum()     # Get total number of current feature
            a+=result[current_class][j]                             # Increment a by total number of current feature
        result[current_class]['total']=a                            # Assign a, will be used in later calculations
    return result                                                   # Return result


x_train_updated=np.array(x_train_updated)                           # Change pandas dataframe to numpy array
d=fit(x_train_updated,y_train)                                      # Fit ur trainig data 

In [9]:
def probability(dictionary, x, current_class): 
    output = ma.log(dictionary[current_class]["total"]) - ma.log(dictionary["total"])
    num_features = len(dictionary[current_class].keys()) - 2
    for j in range( num_features ):
        if x[j]==0:
            continue
        count_current_class_with_value_j = dictionary[current_class][j]+1
        count_current_class = dictionary[current_class]['total']+num_features
        current_j_probablity = ma.log(count_current_class_with_value_j) - ma.log(count_current_class)
        output = output + current_j_probablity
    return output

def doSinglePrediction(x,dictionary):                                  # Function to predict class for a single row
    classes = dictionary.keys()                                        # Get all possible classes
    best_p = -100                                                      # Initialise best probablity & class to some -ve number
    best_class = -100 
    first_run = True                                                   # Running for first time = True
    for current_class in classes:                                      # Iterate over each possible class
        if (current_class == "total"):                                 # Ignore 'total' key
            continue
        p_current_class = probability(dictionary, x, current_class)    # Get probablity for x belonging to current class
        if (first_run or p_current_class > best_p):                    # If this is greatest probablity till now, change the
            best_p = p_current_class                                    # value of greatest probability & predicted class
            best_class = current_class
        first_run = False                                              # First run complete
    return best_class                                                  # Return the predicted class out of all classes

def y_predict(x_test,d,features):                                       # Function to predict the output values for test data
    y_pred=np.zeros(len(x_test))                                        # We will use this numpy array to store predictions
    x_test_updated=updateXTrain(x_test,features)                        # Update x_test so that our model can operate on it
    x_test_updated=np.array(x_test_updated)                             # Change pandas dataframe to numpy array
    for i in range(len(x_test)):                                                                                                        # Extract one-one row from data
        curr_x=x_test_updated[i,:]                                      # Extract one-one row 
        y_pred[i]=doSinglePrediction(curr_x,d)                          # Predict class for current row, store the prediction
    return y_pred                                                       # Return predictions


In [10]:
y_pred=y_predict(x_test,d,features)                                     # Call function to predict classes for x_test
print(classification_report(y_test,y_pred))                             # Print classification report
print(confusion_matrix(y_test,y_pred))                                  # Print confusion matrix


             precision    recall  f1-score   support

          0       0.89      0.82      0.85       132
          1       0.51      0.81      0.62       130
          2       0.67      0.01      0.03       149
          3       0.53      0.80      0.64       160
          4       0.64      0.82      0.72       153
          5       0.88      0.68      0.76       151
          6       0.75      0.83      0.79       141
          7       0.81      0.84      0.83       158
          8       0.91      0.84      0.87       140
          9       0.89      0.91      0.90       161
         10       0.98      0.89      0.93       151
         11       0.95      0.93      0.94       132
         12       0.69      0.76      0.73       139
         13       0.98      0.88      0.93       144
         14       0.94      0.91      0.93       158
         15       0.83      0.94      0.88       139
         16       0.85      0.92      0.89       139
         17       0.96      0.88      0.92   

In [11]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test,y_pred))

0.8002827854365501


In [12]:
print('\n\n\t\tCOMPARING WITH LIBRARY IMPLEMENTATION')      # Compare built model with the sklearn's inbuilt model

from sklearn.naive_bayes import MultinomialNB               # Import MultinomialDB from sklearn
clf = MultinomialNB()                                       # Create object of MultinomialDB
clf.fit(x_train_updated, y_train)                           # Fit the training data
x_test_updated=updateXTrain(x_test,features)                # Update x_test so that model can operate on it
x_test_updated=np.array(x_test_updated)                     # Convert to np array
y_pred = clf.predict(x_test_updated)                        # Do predictions
print(classification_report(y_test,y_pred))                 # Print classification report
print(confusion_matrix(y_test,y_pred))                      # Print confusion matrix
print(accuracy_score(y_test,y_pred))



		COMPARING WITH LIBRARY IMPLEMENTATION
             precision    recall  f1-score   support

          0       0.83      0.83      0.83       132
          1       0.49      0.68      0.57       130
          2       0.40      0.03      0.05       149
          3       0.56      0.80      0.66       160
          4       0.65      0.82      0.73       153
          5       0.60      0.70      0.65       151
          6       0.72      0.79      0.75       141
          7       0.80      0.82      0.81       158
          8       0.76      0.88      0.81       140
          9       0.91      0.92      0.92       161
         10       0.96      0.91      0.94       151
         11       0.98      0.93      0.95       132
         12       0.76      0.75      0.75       139
         13       0.87      0.85      0.86       144
         14       0.96      0.85      0.90       158
         15       0.89      0.89      0.89       139
         16       0.88      0.88      0.88       139
   

On comparing above results we can see that our implementation is performing better than the library implementation of Naive Bayes.